<a href="https://colab.research.google.com/github/cjkreienkamp/wpolo-scoresheet-ocr/blob/main/wpolo_scoresheet_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytesseract opencv-python openpyxl
!sudo apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
import cv2
import numpy as np

In [ ]:
def draw_lines(img, hough_lines, color=(0,0,255), thickness=2):
  if hough_lines is not None:
    for line in hough_lines:
      for rho,theta in line:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))

        cv2.line(img,pt1,pt2,color,thickness)

1. rotate image using lines
2. read in cap data
3. read in team data
4. read in remark
5. read in score
6. read in time
7. read in column
8. read in entire game log
9. locate position on page by comparing it to other game sheets, we should have an expected size for that area

In [ ]:
image = cv2.imread('data/gamelog_capnumber.png')
cv2.imwrite('image.png',image)

gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imwrite('gray_image.png', gray_image)

_, binary_image = cv2.threshold(gray_image, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
cv2.imwrite('binary_image.png', binary_image)

canny_edges = cv2.Canny(binary_image, 100, 200, None, 3)
cv2.imwrite('canny_edges.png',canny_edges)

# difficult to know the 150 value!
hough_lines = cv2.HoughLines(canny_edges, 1 , np.pi/180 , 150)
image_w_lines = image.copy()
draw_lines(image_w_lines, hough_lines)
cv2.imwrite('hough_lines.png',image_w_lines)

True

In [ ]:
angle_sum = 0
angle_average = 0
if hough_lines is not None:
    for line in hough_lines:
      for rho,theta in line:
        angle_offcenter = min(abs(theta), abs(theta - np.pi/2))
        angle_sum += angle_offcenter
    angle_average = angle_sum / len(hough_lines)

(height, width) = image.shape[:2]
center = (width / 2, height / 2)
rotate_matrix = cv2.getRotationMatrix2D(center, angle_average*180/np.pi, 1)

rotated = cv2.warpAffine(image, rotate_matrix, dsize=(width, height))
cv2.imwrite('rotated.png',rotated)

True

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2)) # create a 2x2 kernel
dilate_erode = cv2.erode(binary_image, kernel, iterations=1) # apply the erode function with the kernel
cv2.imwrite('dilate_erode.png',dilate_erode)
text = pytesseract.image_to_string(binary_image)
print(text)

In [ ]:
import pytesseract

image_RGB = cv2.cvtColor(dilate_erode, cv2.COLOR_BGR2RGB)
text = pytesseract.image_to_string(dilate_erode)
print(text)

# Unused Code

In [ ]:


image_w_lines = image.copy()
# how can we choose the best value of 150...higher values make less lines detected
hough_lines = cv2.HoughLinesP(canny_edges, 1, np.pi / 180, 150, None, 150, 10)

for line in hough_lines:
    l = line[0]
    cv2.line(image_w_lines, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv2.LINE_AA)
cv2.imwrite('hough_lines.png',image_w_lines)

In [ ]:
from scipy.ndimage import interpolation as inter

def correct_skew(image, delta=1, limit=5):
    def determine_score(arr, angle):
        data = inter.rotate(arr, angle, reshape=False, order=0)
        histogram = np.sum(data, axis=1, dtype=float)
        score = np.sum((histogram[1:] - histogram[:-1]) ** 2, dtype=float)
        return histogram, score

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    scores = []
    angles = np.arange(-limit, limit + delta, delta)
    for angle in angles:
        histogram, score = determine_score(thresh, angle)
        scores.append(score)

    best_angle = angles[scores.index(max(scores))]

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, best_angle, 1.0)
    corrected = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, \
            borderMode=cv2.BORDER_REPLICATE)

    return best_angle, corrected

In [ ]:
image = cv2.imread('gamelog_1col.png')
angle, corrected = correct_skew(image)
print('Skew angle:', angle)
cv2.imwrite('corrected.png', corrected)

<ipython-input-4-c3ee19f3cc88>:5: DeprecationWarning: Please import `rotate` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  data = inter.rotate(arr, angle, reshape=False, order=0)


Skew angle: 1


True

In [ ]:
def findTiltAngle(image_edges):

  h, theta, d = hough_line(image_edges)
  accum, angles, dists = hough_line_peaks(h, theta, d)
  angle = np.rad2deg(mode(angles)[0][0])

  if (angle < 0):
    angle = angle + 90
  else:
    angle = angle - 90
    return angle

In [ ]:
angle = findTiltAngle(dst3)
print(angle)

IndexError: invalid index to scalar variable.

In [ ]:
mask = binary_image.copy()
height,width = mask.shape
skel = np.zeros([height,width],dtype=np.uint8)      #[height,width,3]
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
temp_nonzero = np.count_nonzero(mask)
while(np.count_nonzero(mask) != 0 ):
    eroded = cv2.erode(mask,kernel)
    cv2.imwrite('eroded.png',eroded)
    temp = cv2.dilate(eroded,kernel)
    cv2.imwrite('dilate.png',temp)
    temp = cv2.subtract(mask,temp)
    skel = cv2.bitwise_or(skel,temp)
    mask = eroded.copy()

cv2.imwrite('skel.png',skel)

In [ ]:
dst1 = cv2.Canny(image, 50, 200, None, 3)
cv2.imwrite('canny1.png',dst1)
linesP1 = cv2.HoughLinesP(dst1, 1, np.pi / 180, 50, None, 50, 10)
cdstP1 = image.copy()
if linesP1 is not None:
        for i in range(0, len(linesP1)):
            l = linesP1[i][0]
            cv2.line(cdstP1, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv2.LINE_AA)
cv2.imwrite('lines1.png',cdstP1)

dst2 = cv2.Canny(gray_image, 100, 200, None, 3)
cv2.imwrite('canny2.png',dst2)
linesP2 = cv2.HoughLinesP(dst2, 1, np.pi / 180, 50, None, 50, 10)
cdstP2 = image.copy()
if linesP2 is not None:
        for i in range(0, len(linesP2)):
            l = linesP2[i][0]
            cv2.line(cdstP2, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv2.LINE_AA)
cv2.imwrite('lines2.png',cdstP2)

dst3 = cv2.Canny(binary_image, 100, 200, None, 3)
cv2.imwrite('canny3.png',dst3)
linesP3 = cv2.HoughLinesP(dst3, 1, np.pi / 180, 50, None, 50, 10)
cdstP3 = image.copy()
if linesP3 is not None:
        for i in range(0, len(linesP3)):
            l = linesP3[i][0]
            cv2.line(cdstP3, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv2.LINE_AA)
cv2.imwrite('lines3.png',cdstP3)

dst4 = cv2.Canny(skel, 100, 200, None, 3)
cv2.imwrite('canny4.png',dst4)
linesP4 = cv2.HoughLinesP(dst4, 1, np.pi / 180, 50, None, 50, 10)
cdstP4 = image.copy()
if linesP4 is not None:
        for i in range(0, len(linesP4)):
            l = linesP4[i][0]
            cv2.line(cdstP4, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv2.LINE_AA)
cv2.imwrite('lines4.png',cdstP4)

In [ ]:
edges = cv2.Canny(binary_image,100,200)
result = image.copy()
cv2.imwrite('edges.png',edges)
lines = cv2.HoughLinesP(edges,1,np.pi/180,40,minLineLength=30,maxLineGap=30)
cv2.imwrite('lines.png',lines)
i = 0
for x1,y1,x2,y2 in lines[0]:
    i+=1
    cv2.line(result,(x1,y1),(x2,y2),(255,0,0),1)

cv2.imwrite('result.png',result)

In [ ]:

lines_edges = cv2.addWeighted(image, 0.8, result, 1, 0)
cv2.imwrite('lines_edges.png',lines_edges)

In [ ]:
from pytesseract import image_to_string
extracted_text = image_to_string(binary_image)
with open('output.txt', 'w') as file:
    file.write(extracted_text)

In [ ]:
# Load image, grayscale, Otsu's threshold
image = cv2.imread('gamelog_1col.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = 255 - gray
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

# Compute rotated bounding box
coords = np.column_stack(np.where(thresh > 0))
angle = cv2.minAreaRect(coords)[-1]

if angle < -45:
    angle = -(90 + angle)
else:
    angle = -angle
print(angle)

# Rotate image to deskew
(h, w) = image.shape[:2]
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, angle, 1.0)
rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

cv2.imwrite('thresh.png', thresh)
cv2.imwrite('rotated.png', rotated)

-90.0


True

In [ ]:
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage.transform import rotate
from skimage.transform import (hough_line, hough_line_peaks)
from scipy.stats import mode
from skimage import io
from skimage.filters import threshold_otsu, sobel
from matplotlib import cm